In [ ]:
import re
import os
import string
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
import datetime
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization, Input, Embedding, SimpleRNN, Dense, Bidirectional, LSTM, Dropout, GRU, Conv1D, Flatten
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
import gensim.downloader as api
from tensorboard.plugins import projector

In [ ]:
train_ds, val_ds, test_ds = tfds.load(
    "imdb_reviews",
    split=["train", "test[:50%]", "test[50%:]"],
    as_supervised=True
)

In [ ]:
train_ds

In [ ]:
for review, label in train_ds.take(2):
  print(review)
  print(label)

In [ ]:
def standardization(input_data):
  lower_case = tf.strings.lower(input_data)
  no_tag = tf.strings.regex_replace(lower_case, "<[^>]+>", "") # remove html tag from text
  output = tf.strings.regex_replace(no_tag, "[%s]"%re.escape(string.punctuation), "") # remove punctuation

  return output

In [ ]:
VOCB_SIZE = 10000
SEQUENCE_LENGTH = 250
BATCH_SIZE = 64
EMBEDING_DIM = 300

In [ ]:
vectorize_layer = TextVectorization(
    standardize=standardization,
    max_tokens=VOCB_SIZE,
    output_mode="int",
    output_sequence_length=SEQUENCE_LENGTH
)

In [ ]:
# lengths = []
# words = []

# for review, label in train_ds.take(10):
#   for word in tf.strings.split(review, sep=" "):
#     if word in words:
#       pass
#     else:
#       words.append(word)
#   lengths.append(len(tf.strings.split(review, sep=" ")))

In [ ]:
training_data = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(training_data)

In [ ]:
vectorize_layer.get_vocabulary()

In [ ]:
len(vectorize_layer.get_vocabulary())

In [ ]:
for review, label in train_ds.take(1):
  print(review)
  print(label)

In [ ]:
def vectorizer(review, label):
  return vectorize_layer(review), label

In [ ]:
train_dataset = train_ds.map(vectorizer)
val_dataset = val_ds.map(vectorizer)

In [ ]:
for review, label in train_dataset.take(1):
  print(review)
  print(label)

In [ ]:
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
EMBEDING_DIM = 64
model = tf.keras.Sequential([
    Input(shape=(SEQUENCE_LENGTH,)),
    Embedding(VOCB_SIZE, EMBEDING_DIM),
    SimpleRNN(32),
    Dense(1, activation="sigmoid")
])

model.summary()

In [ ]:
EMBEDING_DIM = 64
model = tf.keras.Sequential([
    Input(shape=(SEQUENCE_LENGTH,)),
    Embedding(VOCB_SIZE, EMBEDING_DIM),

    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(32)),

    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

model.summary()

In [ ]:
EMBEDING_DIM = 64
model = tf.keras.Sequential([
    Input(shape=(SEQUENCE_LENGTH,)),
    Embedding(VOCB_SIZE, EMBEDING_DIM),

    Bidirectional(GRU(64, return_sequences=True)),
    Bidirectional(GRU(32)),

    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

model.summary()

In [ ]:
EMBEDDING_DIM=300
model=tf.keras.models.Sequential([
    Input(shape=(SEQUENCE_LENGTH,)),
    Embedding(VOCB_SIZE,EMBEDDING_DIM),

    Conv1D(32, 3, activation='relu',),
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1,activation='sigmoid'),
])
model.summary()

In [ ]:
checkpoint_filepath = '/content/rnn.weights.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
log_dir = 'logs/imdb/fit'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '/'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=5, callbacks=[tensorboard_callback])

In [ ]:
with open(os.path.join(log_dir,'metadata.tsv'),"w",encoding="utf-8") as f:
  for i in range(VOCB_SIZE):
    f.write("{} {}\n".format(i,vectorize_layer.get_vocabulary()[i]))

In [ ]:
embedding_weights=tf.Variable(model.layers[0].get_weights()[0])
print(embedding_weights.shape)

In [ ]:
checkpoint=tf.train.Checkpoint(embedding=embedding_weights)
checkpoint.save(os.path.join(log_dir,"embedding.ckpt"))

config=projector.ProjectorConfig()
embedding=config.embeddings.add()

In [ ]:
embedding.metadata_path='metadata.tsv'
projector.visualize_embeddings(log_dir,config)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/imdb/fit/

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("model_loss")
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['tain', "val"], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("model_accuracy")
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['tain', "val"], loc='upper left')
plt.show()

In [ ]:
word2vec = api.load("word2vec-google-news-300")

In [ ]:
word2vec.vectors.shape

In [ ]:
word2vec.key_to_index

In [ ]:
len(word2vec["The"])

In [ ]:
word2vec.most_similar('Man')

In [ ]:
def first_cap(word):
  return word[0].upper() + word[1:]

In [ ]:
pretrained_embedding = []
for i in range(len(vectorize_layer.get_vocabulary())):
 try:
  pretrained_embedding.append(word2vec[vectorize_layer.get_vocabulary()[i]])
 except:
  print(vectorize_layer.get_vocabulary()[i])
  try:
    pretrained_embedding.append(word2vec[first_cap(vectorize_layer.get_vocabulary()[i])])
    print("toupper")
  except:
    print("nosloution")
    pretrained_embedding.append(random.normal(loc=0,scale=1,size=(EMBEDING_DIM,)))

  if i%1000==0:
    print("i is=== ", i)

In [ ]:
pretrained_embedding_array = np.array(pretrained_embedding)

In [ ]:
EMBEDING_DIM = 300
model = tf.keras.Sequential([
    Input(shape=(SEQUENCE_LENGTH,)),
    Embedding(VOCB_SIZE, EMBEDING_DIM, embeddings_initializer=tf.keras.initializers.Constant(pretrained_embedding), trainable=True),

    Conv1D(32, 3, activation="relu"),
    Flatten(),

    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history=model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,)

In [ ]:
test_data=tf.data.Dataset.from_tensor_slices([["this movie looks very interesting, i love the fact that the actors do a great job in showing how people lived in the 18th century, which wasn't very good at all. But atleast this movie recreates this scenes! "],
                                              ["very good start, but movie started becoming interesting at some point and fortunately at some point it started becoming much more fun, though there was too much background noise, so in all i liked this movie "],])


In [ ]:
def vectorizer_test(review):
    return vectorize_layer(review)
test_dataset=test_data.map(vectorizer_test)

In [ ]:
model.predict(test_dataset)

In [ ]:
inputs = Input(shape=(1,), dtype="string")
vectorized_inputs=vectorize_layer(inputs)
outputs = model(vectorized_inputs)
inference_ready_model = tf.keras.Model(inputs, outputs)
inference_ready_model.summary()

In [ ]:
inference_ready_model.predict(["this movie looks very interesting, i love the fact that the actors do a great job in showing how people lived in the 18th century, which wasn't very good at all. But atleast this movie recreates this scenes! ",
                               "very good start, but movie started becoming interesting at some point and fortunately at some point it started becoming much more fun, though there was too much background noise, so in all i liked this movie "])